In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results): 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=1, circle_radius=1)
                             )
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=1, circle_radius=1)
                             )
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=1, circle_radius=1)
                             )

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [11]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('test-videos') 

parent_folder = 'videos'
# Actions that we try to detect
actions = np.array([name for name in os.listdir(parent_folder) if os.path.isdir(os.path.join(parent_folder, name))])

# Thirty videos worth of data
no_sequences = 5

# Videos are going to be 30 frames in length
sequence_length = 50

In [9]:
cap = cv2.VideoCapture(0,cv2.CAP_DSHOW)
W, H = 640, 480
cap.set(cv2.CAP_PROP_FRAME_WIDTH, W)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, H)
cap.set(cv2.CAP_PROP_FPS, 30)
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'))
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Show to screen
        cv2.imshow('OpenCV Feed', frame)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [13]:
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
W, H = 640, 480
cap.set(cv2.CAP_PROP_FRAME_WIDTH, W)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, H)
cap.set(cv2.CAP_PROP_FPS, 30)
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'))
# Set mediapipe model 
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Define video writer for the current sequence
            directory_path = os.path.join(DATA_PATH, action)
            os.makedirs(directory_path, exist_ok=True)
            video_filename = os.path.join(DATA_PATH, action, f"{sequence}.mp4")
            print(video_filename)
            video_writer = cv2.VideoWriter(video_filename, cv2.VideoWriter_fourcc(*'XVID'), 30, (W, H))

            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):
                # Read feed
                ret, frame = cap.read()

                # Apply wait logic
                if frame_num == 0:
                    cv2.putText(frame, 'STARTING COLLECTION', (120, 200),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                cv2.putText(frame, f'Collecting frames for {action} Video Number {sequence}', (15, 12),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

                # Write frame to video
                video_writer.write(frame)

                # Show frame to screen
                cv2.imshow('OpenCV Feed', frame)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

            # Release the video writer
            video_writer.release()
            cv2.waitKey(1000)

        # Release the video capture
    cap.release()
    cv2.destroyAllWindows()

test-videos\fine\0.mp4
test-videos\fine\1.mp4
test-videos\finish\0.mp4
test-videos\finish\1.mp4
test-videos\go_to\0.mp4
test-videos\go_to\1.mp4
test-videos\how\0.mp4
test-videos\how\1.mp4
test-videos\i\0.mp4
test-videos\i\1.mp4
test-videos\i_love_you\0.mp4
test-videos\i_love_you\1.mp4
test-videos\learn\0.mp4
test-videos\learn\1.mp4
test-videos\more\0.mp4
test-videos\more\1.mp4
test-videos\nice_to_meet_you\0.mp4
test-videos\nice_to_meet_you\1.mp4
test-videos\please\0.mp4
test-videos\please\1.mp4
test-videos\see_you_later\0.mp4
test-videos\see_you_later\1.mp4
test-videos\thanks\0.mp4
test-videos\thanks\1.mp4
test-videos\want\0.mp4
test-videos\want\1.mp4
test-videos\yes\0.mp4
test-videos\yes\1.mp4
